# 參考資源

[Beautiful Soup 基本用法](https://blog.gtwang.org/programming/python-beautiful-soup-module-scrape-web-pages-tutorial/)

[Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)

## 基本用法

In [1]:
# 引入 Beautiful Soup 模組
from bs4 import BeautifulSoup

# 原始 HTML 程式碼
html_doc = """
<html><head><title>Hello World</title></head>
<body><h2>Test Header</h2>
<p>This is a test.</p>
<a id="link1" href="/my_link1">Link 1</a>
<a id="link2" href="/my_link2">Link 2</a>
<p>Hello, <b class="boldtext">Bold Text</b></p>
</body></html>
"""

# 以 Beautiful Soup 解析 HTML 程式碼
soup = BeautifulSoup(html_doc, 'html.parser')
# 備註: 'html.parser' : 执行速度适中，文档容错能力强
#       "lxml" : 速度快

In [5]:
soup
# 輸出排版後的 HTML 程式碼
print('-' * 30)
print(soup.prettify())


<html><head><title>Hello World</title></head>
<body><h2>Test Header</h2>
<p>This is a test.</p>
<a href="/my_link1" id="link1">Link 1</a>
<a href="/my_link2" id="link2">Link 2</a>
<p>Hello, <b class="boldtext">Bold Text</b></p>
</body></html>

------------------------------
<html>
 <head>
  <title>
   Hello World
  </title>
 </head>
 <body>
  <h2>
   Test Header
  </h2>
  <p>
   This is a test.
  </p>
  <a href="/my_link1" id="link1">
   Link 1
  </a>
  <a href="/my_link2" id="link2">
   Link 2
  </a>
  <p>
   Hello,
   <b class="boldtext">
    Bold Text
   </b>
  </p>
 </body>
</html>



### 取得節點文字內容

In [7]:
# 網頁標題 HTML 標籤
title_tag = soup.title
print(title_tag)

# 網頁的標題文字
print(title_tag.string)

<title>Hello World</title>
Hello World


### 搜尋節點

In [8]:
# 所有的超連結
a_tags = soup.find_all('a')
for tag in a_tags:
  # 輸出超連結的文字
  print(tag.string)

Link 1
Link 2


### 取出節點屬性

In [9]:
for tag in a_tags:
  # 輸出超連結網址
  print(tag.get('href'))

/my_link1
/my_link2


### 同時搜尋多種標籤

In [10]:
# 搜尋所有超連結與粗體字
tags = soup.find_all(["a", "b"])
print(tags)

[<a href="/my_link1" id="link1">Link 1</a>, <a href="/my_link2" id="link2">Link 2</a>, <b class="boldtext">Bold Text</b>]


### 限制搜尋節點數量

In [12]:
# 限制搜尋結果數量
tags = soup.find_all(["a", "b"], limit=2)
print(tags)

print('-' * 30)

# 只抓出第一個符合條件的節點
a_tag = soup.find("a")
print(a_tag)

[<a href="/my_link1" id="link1">Link 1</a>, <a href="/my_link2" id="link2">Link 2</a>]
------------------------------
<a href="/my_link1" id="link1">Link 1</a>


### 遞迴搜尋

In [13]:
# 預設會以遞迴搜尋
soup.html.find_all("title")

# 不使用遞迴搜尋，僅尋找次一層的子節點
soup.html.find_all("title", recursive=False)

[<title>Hello World</title>]

[]

## 搜尋

### 以 HTML 屬性搜尋

In [15]:
# 根據 id 搜尋
link2_tag = soup.find(id='link2')
print(link2_tag)

<a href="/my_link2" id="link2">Link 2</a>


In [16]:
# 搜尋 href 屬性為 /my_link1 的 a 節點
a_tag = soup.find_all("a", href="/my_link1")
print(a_tag)

[<a href="/my_link1" id="link1">Link 1</a>]


In [17]:
#搜尋屬性時，也可以使用正規表示法
#例如以正規表示法比對超連結網址：

import re

# 以正規表示法比對超連結網址
links = soup.find_all(href=re.compile("^/my_link\d"))
print(links)

[<a href="/my_link1" id="link1">Link 1</a>, <a href="/my_link2" id="link2">Link 2</a>]


In [19]:
# 以多個屬性條件來篩選
link = soup.find_all(href=re.compile("^/my_link\d"), id="link1")
print(link)

[<a href="/my_link1" id="link1">Link 1</a>]


在 HTML5 中有一些屬性名稱若直接寫在 Python 的參數中會有一些問題，

例如 data-* 這類的屬性直接寫的話，就會產生錯誤訊息：

In [20]:
data_soup = BeautifulSoup('<div data-foo="value">foo!</div>', 'html.parser')

# 錯誤的用法
data_soup.find_all(data-foo="value")

SyntaxError: keyword can't be an expression (<ipython-input-20-ee1f74febb50>, line 4)

遇到這種狀況，可以把屬性的名稱與值放進一個 dictionary 中，

再將此 dictionary 指定給 attrs 參數即可：

In [22]:
data_soup = BeautifulSoup('<div data-foo="value">foo!</div>', 'html.parser')

# 正確的用法
data_soup.find_all(attrs={"data-foo": "value"})

[<div data-foo="value">foo!</div>]

###  以 CSS 搜尋

In [23]:
# 搜尋 class 為 boldtext 的 b 節點
b_tag = soup.find_all("b", class_="boldtext")
print(b_tag)

[<b class="boldtext">Bold Text</b>]


In [24]:
# 以正規表示法搜尋 class 屬性
b_tag = soup.find_all(class_=re.compile("^bold"))
print(b_tag)

[<b class="boldtext">Bold Text</b>]


In [25]:
css_soup = BeautifulSoup('<p class="body strikeout"></p>', 'html.parser')

# 只要其中一個 class 符合就算比對成功
p_tag = css_soup.find_all("p", class_="strikeout")
print(p_tag)

[<p class="body strikeout"></p>]


In [26]:
# 若順序不同，則會失敗
p_tag = css_soup.find_all("p", class_="strikeout body")
print(p_tag)

[]


### 以文字內容搜尋

In [27]:
links_html = """
<a id="link1" href="/my_link1">Link One</a>
<a id="link2" href="/my_link2">Link Two</a>
<a id="link3" href="/my_link3">Link Three</a>
"""
soup = BeautifulSoup(links_html, 'html.parser')

# 搜尋文字為「Link One」的超連結
soup.find_all("a", string="Link One")

[<a href="/my_link1" id="link1">Link One</a>]

In [28]:
# 以正規表示法搜尋文字為「Link」開頭的超連結
soup.find_all("a", string=re.compile("^Link"))

[<a href="/my_link1" id="link1">Link One</a>,
 <a href="/my_link2" id="link2">Link Two</a>,
 <a href="/my_link3" id="link3">Link Three</a>]

### 向上、向前與向後搜尋

In [29]:
html_doc = """
<body><p class="my_par">
<a id="link1" href="/my_link1">Link 1</a>
<a id="link2" href="/my_link2">Link 2</a>
<a id="link3" href="/my_link3">Link 3</a>
<a id="link3" href="/my_link4">Link 4</a>
</p></body>
"""
soup = BeautifulSoup(html_doc, 'html.parser')
link2_tag = soup.find(id="link2")

# 往上層尋找 p 節點
p_tag = link2_tag.find_parents("p")
print(p_tag)

[<p class="my_par">
<a href="/my_link1" id="link1">Link 1</a>
<a href="/my_link2" id="link2">Link 2</a>
<a href="/my_link3" id="link3">Link 3</a>
<a href="/my_link4" id="link3">Link 4</a>
</p>]


In [30]:
# 在同一層往前尋找 a 節點
link_tag = link2_tag.find_previous_siblings("a")
print(link_tag)

[<a href="/my_link1" id="link1">Link 1</a>]


In [31]:
# 在同一層往後尋找 a 節點
link_tag = link2_tag.find_next_siblings("a")
print(link_tag)

[<a href="/my_link3" id="link3">Link 3</a>, <a href="/my_link4" id="link3">Link 4</a>]


## 網頁檔案

In [ ]:
from bs4 import BeautifulSoup
# 從檔案讀取 HTML 程式碼進行解析
with open("index.html") as f:
    soup = BeautifulSoup(f)

# 實例

## 下載 Yahoo 頭條新聞

In [32]:
import requests
from bs4 import BeautifulSoup

# 下載 Yahoo 首頁內容
r = requests.get('https://tw.yahoo.com/')

# 確認是否下載成功
if r.status_code == requests.codes.ok:
    # 以 BeautifulSoup 解析 HTML 程式碼
    soup = BeautifulSoup(r.text, 'html.parser')

    # 以 CSS 的 class 抓出各類頭條新聞
    stories = soup.find_all('a', class_='story-title')
    for s in stories:
        # 新聞標題
        print("標題：" + s.text)
        # 新聞網址
        print("網址：" + s.get('href'))
        
##程式執行之後，就會輸出 Yahoo 首頁頭條新聞的標題與網址：

標題：她身價277億「被一個蛋打敗」
網址：https://tw.news.yahoo.com/%E7%BE%8E%E5%9C%8B%E7%B6%B2%E7%B4%85ig%E6%8C%89%E8%AE%9A%E6%95%B8%E8%A2%AB%E7%A0%B4-%E5%B0%8D%E6%89%8B%E7%AB%9F%E6%98%AF%E4%B8%80%E9%A1%86%E8%9B%8B-024243735.html
標題：曾上節目泣訴 1個月後她被刺死
網址：https://tw.news.yahoo.com/%E5%AE%B6%E6%9A%B4%E4%BB%A4%E7%84%A1%E7%94%A8-%E5%A5%B3%E6%9B%BE%E4%B8%8A%E4%BA%8E%E7%BE%8E%E4%BA%BA%E7%AF%80%E7%9B%AE%E6%B3%A3%E8%A8%B4-1%E5%80%8B%E6%9C%88%E5%BE%8C%E8%A2%AB%E5%89%8D%E5%A4%AB%E5%88%BA%E6%AD%BB-145102951.html
標題：夫領無薪 她隨手一買進帳300萬
網址：https://tw.news.yahoo.com/%E4%B8%88%E5%A4%AB%E6%98%AF%E6%94%BF%E5%BA%9C%E5%81%9C%E6%93%BA%E5%8F%97%E5%AE%B3%E8%80%85-%E7%BE%8E%E5%A9%A6%E4%BA%BA%E8%B2%B7%E5%BD%A9%E5%88%B8%E4%B8%AD311%E8%90%AC-000839056.html
標題：2200萬買到3千萬車 付完錢傻了
網址：https://tw.news.yahoo.com/%E5%B0%88%E5%9D%91%E9%A0%AD%E5%AE%B6-%E7%9B%B4%E6%92%AD%E8%B6%85%E8%B7%91%E5%8D%8A%E5%83%B9%E8%B3%A3-%E5%AF%8C%E6%AF%94%E5%A3%AB%E5%B9%B3%E5%8F%B0%E7%84%A1%E8%B3%B4%E5%90%B8%E9%87%9110%E5%84%84%E5%85%83-225859206.html
標題：報喜！一姐發功 輕鬆寫意過

## 下載 Google 搜尋結果

In [34]:
import requests
from bs4 import BeautifulSoup

# Google 搜尋 URL
google_url = 'https://www.google.com.tw/search'

# 查詢參數
my_params = {'q': '寒流'}

# 下載 Google 搜尋結果
r = requests.get(google_url, params = my_params)

# 確認是否下載成功
if r.status_code == requests.codes.ok:
    # 以 BeautifulSoup 解析 HTML 原始碼
    soup = BeautifulSoup(r.text, 'html.parser')

    # 觀察 HTML 原始碼
    # print(soup.prettify())

    # 以 CSS 的選擇器來抓取 Google 的搜尋結果
    items = soup.select('div.g > h3.r > a[href^="/url"]')
    for i in items:
        # 標題
        print("標題：" + i.text)
        # 網址
        print("網址：" + i.get('href'))

標題：氣象局一張圖來了！ 未來一周降溫但寒流不會來| 生活新聞| 生活| 聯合 ...
網址：/url?q=https://udn.com/news/story/7266/3514053&sa=U&ved=0ahUKEwix_svQpu_fAhWCa7wKHf9OCTQQFggcMAM&usg=AOvVaw3D99X057WcYckBOCS93dT0
標題：冬天終於來了！氣象局預測：首波寒流最快12月中旬報到| 生活| 新頭殼 ...
網址：/url?q=https://newtalk.tw/news/view/2018-11-30/174090&sa=U&ved=0ahUKEwix_svQpu_fAhWCa7wKHf9OCTQQFggiMAQ&usg=AOvVaw0pMV8mMn0L9BPXNB4ohGY1
標題：寒流 - 中央氣象局
網址：/url?q=https://www.cwb.gov.tw/V7/climate/climate_info/taiwan_climate/taiwan_3/taiwan_3_5.html&sa=U&ved=0ahUKEwix_svQpu_fAhWCa7wKHf9OCTQQFggoMAU&usg=AOvVaw3XJq8Dtx9hBd6HRLx4ejYN
標題：第一波寒流時間曝光！氣象局：暖冬仍降10度以下  TVBS新聞網
網址：/url?q=https://news.tvbs.com.tw/life/1039423&sa=U&ved=0ahUKEwix_svQpu_fAhWCa7wKHf9OCTQQFggtMAY&usg=AOvVaw2iobkbt8eIAx-w_KMPC7ws
標題：羽絨外套準備！首波寒流時間曝光氣象局：低溫恐10度- Yahoo奇摩新聞
網址：/url?q=https://tw.news.yahoo.com/%25E7%25BE%25BD%25E7%25B5%25A8%25E5%25A4%2596%25E5%25A5%2597%25E6%25BA%2596%25E5%2582%2599-%25E9%25A6%2596%25E6%25B3%25A2%25E5%25AF%2592%25E6%25B5%2581%25E8%25A6%2581%25E4%25BE%2586%25E4%25BA%2586-%25E6%